Meta-analyses of clinical trials often treat the number of patients experiencing a medical event as <mark>binomially distributed when individual patient data for fitting standard time-to-event models are unavailable</mark>. Assuming <mark>identical drop-out time distributions across arms, random censorship and low proportions of patients with an event</mark>, a binomial approach results in a valid test of the null hypothesis of no treatment effect with minimal loss in efficiency compared to time-to-event methods. To deal with differences in follow-up - at the cost of assuming specific distributions for event and drop-out times - we propose a <mark>hierarchical multivariate meta-analysis model using the aggregate data likelihood based on the number of cases, fatal cases and discontinuations in each group, as well as the planned trial duration and groups sizes</mark>. Such a model also enables <mark>exchangeability assumptions about parameters of survival distributions, for which they are more appropriate than for the expected proportion of patients with an event across trials of substantially different length</mark>. Borrowing information from other trials within a meta-analysis or from historical data is particularly useful for rare events data. Prior information or exchangeability assumptions also avoid the parameter identifiability problems that arise when using more flexible event and drop-out time distributions than the exponential one. We discuss the derivation of robust historical priors and illustrate the discussed methods using an example. We also compare the proposed approach against other aggregate data meta-analysis methods in a simulation study.

Source: Kaggle


<u>This notebook was written with the help of ChatGPT, with the purpose to learn about how to approach a complex data and developing a machine learning model. The project assists with learning more about the concepts used for multivariate studies.</u>

In [5]:
#Checking if the data loaded correctly
# Using the column analysis, check if each column has null values or missing values
# It seems like the z and m columns have null values but with a placeholder . instead of an empty column
df = spark.sql("SELECT * FROM testing_lakehouse.`meta-analysis medical events dataset` LIMIT 1000")
display(df)

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a72a08ac-1ea3-4889-bd13-f4e46f15969c)

# In this section I am exploring and studying and cleaning the data.

In [6]:
# checking for null values which have a . as a placeholder 
null_rows = spark.sql("SELECT * FROM testing_lakehouse.`meta-analysis medical events dataset` WHERE z = '.' OR m = '.'")
display(null_rows)

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2b8c413f-9ba2-4ea0-a89b-c8bb2ee8723c)

Between z and m there are 10 rows of data with null values out of 172 rows

In [7]:
#removing the null values from the data and assigning them to a clean_table
clean_table = spark.sql("DELETE FROM testing_lakehouse.`meta-analysis medical events dataset` WHERE z = '.' OR m = '.'")
display(clean_table)

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5847ef0c-4922-4b47-b019-0a951d7b0326)

In [8]:
#check if the  data cleaning was effective
df = spark.sql("SELECT * FROM testing_lakehouse.`meta-analysis medical events dataset`")
display(df)

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0a127b19-16a2-4f42-82cf-b6279b65654a)

Now the cleaned data is ready to be used for further analysis

Now we have 162 rows of data to analyse

Here is what we will be looking at

- 	Comparing treatment (test = 1) and control (test = 0) groups.
- 	Analyzing the effect of tau (trial duration) on outcomes.
- 	Evaluating relationships between n (participants), y (events), and z (dropouts).

In [9]:
# Total events and participants in treatment vs. control groups using python
df.groupBy("test").agg(
    {"y": "sum", "n": "sum"}
).withColumnRenamed("sum(y)", "Total_Events").withColumnRenamed("sum(n)", "Total_Participants").show()

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 11, Finished, Available, Finished)

+----+------------+------------------+
|test|Total_Events|Total_Participants|
+----+------------+------------------+
|   1|          82|             10226|
|   0|         155|             20860|
+----+------------+------------------+



We see that there were lower events with participants that received treatment, which shows the effectiveness of the treatments, but it is worth noting the participants who did not receive treatment are almost double in n

In [10]:
# Average events and participants per trial duration
df.groupBy("tau").agg(
    {"y": "avg", "n": "avg"}
).withColumnRenamed("avg(y)", "Avg_Events").withColumnRenamed("avg(n)", "Avg_Participants").orderBy("tau").show()

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 12, Finished, Available, Finished)

+---+-------------------+------------------+
|tau|         Avg_Events|  Avg_Participants|
+---+-------------------+------------------+
|  6|                0.0|              41.0|
|  8|                0.5|            127.75|
| 12|0.16666666666666666|63.916666666666664|
| 16|0.29411764705882354| 78.41176470588235|
| 18| 0.3333333333333333|150.33333333333334|
| 20|                1.0|             112.0|
| 24| 0.9736842105263158|208.26315789473685|
| 26|  1.380952380952381|175.47619047619048|
| 28|                0.0|             316.5|
| 30|                2.0|             518.0|
| 32| 1.2857142857142858|234.14285714285714|
| 40|                0.0|             248.0|
| 44|                3.0|             621.0|
| 50|                2.0|             572.0|
| 52|                2.5|201.58333333333334|
| 54|                1.5|             184.5|
| 56|                3.0|             251.0|
| 72|                4.0|             250.5|
| 78|               11.0|             334.0|
|104|  4.8

We see that with the increase in trial duration the more events occur except at 78 which is an outlier.

display(spark.sql("SELECT sum(n), avg(tau) AS avg_tau FROM testing_lakehouse.`meta-analysis medical events dataset` GROUP BY tau"))

In [11]:
# Calculate dropout rate as a new column
from pyspark.sql.functions import col

df = df.withColumn("dropout_rate", (col("z") / col("n")).alias("dropout_rate"))

# Average dropout rate by treatment group
df.groupBy("test").agg(
    {"dropout_rate": "avg"}
).withColumnRenamed("avg(dropout_rate)", "Avg_Dropout_Rate").show()

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 13, Finished, Available, Finished)

+----+-------------------+
|test|   Avg_Dropout_Rate|
+----+-------------------+
|   1|0.16280074021812715|
|   0| 0.2418862765139815|
+----+-------------------+



We see that the subjects that do not receive treatment are more likely to dropout of the clinical trials.

In [14]:
# Add event rate column
df = df.withColumn("event_rate", col("y") / col("n"))

# Average event rate by treatment group
df.groupBy("test").agg(
    {"event_rate": "avg"}
).withColumnRenamed("avg(event_rate)", "Avg_Event_Rate").show()

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 16, Finished, Available, Finished)

+----+--------------------+
|test|      Avg_Event_Rate|
+----+--------------------+
|   1|0.008875048333764837|
|   0|0.006835306863172582|
+----+--------------------+



In [24]:
# Register the DataFrame as a temporary SQL table
df.createOrReplaceTempView("clinical_data")

# Query to calculate average event rate by trial duration
spark.sql("""
    SELECT tau, AVG(event_rate) AS Avg_Event_Rate
    FROM clinical_data
    GROUP BY tau
    ORDER BY tau
""").show()

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 26, Finished, Available, Finished)

+---+--------------------+
|tau|      Avg_Event_Rate|
+---+--------------------+
|  6|                 0.0|
|  8|0.002986752872553498|
| 12|0.006994047619047...|
| 16|0.003923720687170849|
| 18|0.001872659176029...|
| 20|0.008928571428571428|
| 24|0.004683789898685848|
| 26|0.008124729007645697|
| 28|                 0.0|
| 30|0.003861003861003861|
| 32|0.006350873495222583|
| 40|                 0.0|
| 44|0.004830917874396135|
| 50|0.003496503496503...|
| 52|0.014491684195686584|
| 54|0.005791505791505791|
| 56| 0.01195219123505976|
| 72|0.015374975901291691|
| 78| 0.03299057796265251|
|104| 0.01891491598815829|
+---+--------------------+
only showing top 20 rows



As we can see, the longer the trial the more likely an event occuring, now we are going to build a regression model to measure the strength and direction of the relationship between each feature and the outcome (y)

In [29]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType, DoubleType

#Casting column z to an interger
df = df.withColumn("z", col("z").cast(IntegerType()))

# Prepare features for modeling
assembler = VectorAssembler(
    inputCols=["tau", "n", "z", "test"], outputCol="features"
)
df_model = assembler.transform(df).select("features", "y")

# Train logistic regression model
lr = LogisticRegression(featuresCol="features", labelCol="y")
lr_model = lr.fit(df_model)

# View model coefficients
print("Coefficients:", lr_model.coefficientMatrix)
print("Intercept:", lr_model.interceptVector)

StatementMeta(, 071681ea-0d5d-47f3-81b3-bdef3c872814, 31, Finished, Available, Finished)

Coefficients: DenseMatrix([[-5.35379433e-02, -3.41030734e-03, -3.21874465e-02,
               1.35828753e+00],
             [-8.03714620e-02, -4.66514102e-03,  6.68004404e-04,
               2.74361623e+00],
             [-2.02686680e-02,  1.62860377e-03,  6.66660019e-03,
               1.98090964e+00],
             [-4.52974434e-02, -5.28899139e-03,  4.06365555e-02,
               2.92839370e+00],
             [-9.86550292e-02,  2.34735965e-03,  1.53909957e-02,
               3.12160573e+00],
             [-6.38036849e-03, -1.36728370e-02,  1.92091920e-02,
               4.29377426e+00],
             [-1.70347087e-02, -4.76173412e-03,  3.66602738e-02,
              -1.86905857e+01],
             [-3.08179557e-02,  9.94628039e-03, -2.52161090e-04,
              -2.31198473e+01],
             [ 9.43508040e-03, -4.73794443e-02,  1.22761246e-01,
               1.85850760e+01],
             [ 4.74226657e-02,  1.41881147e-02, -4.03754187e-02,
              -1.62990409e+01],
             [-4

In [4]:
#testing the model, manually
import numpy as np

# Defining the coefficient matrix and intercept vector
coefficients = np.array([
    [-5.35379433e-02, -3.41030734e-03, -3.21874465e-02,  1.35828753e+00],
    [-8.03714620e-02, -4.66514102e-03,  6.68004404e-04,  2.74361623e+00],
    [-2.02686680e-02,  1.62860377e-03,  6.66660019e-03,  1.98090964e+00],
    [-4.52974434e-02, -5.28899139e-03,  4.06365555e-02,  2.92839370e+00],
    [-9.86550292e-02,  2.34735965e-03,  1.53909957e-02,  3.12160573e+00],
    [-6.38036849e-03, -1.36728370e-02,  1.92091920e-02,  4.29377426e+00],
    [-1.70347087e-02, -4.76173412e-03,  3.66602738e-02, -1.86905857e+01],
    [-3.08179557e-02,  9.94628039e-03, -2.52161090e-04, -2.31198473e+01],
    [ 9.43508040e-03, -4.73794443e-02,  1.22761246e-01,  1.85850760e+01],
    [ 4.74226657e-02,  1.41881147e-02, -4.03754187e-02, -1.62990409e+01],
    [-4.02209049e-02, -5.86718581e-03, -7.08945829e-03, -1.31263688e-02],
    [ 2.49491774e-01,  5.16372234e-02, -3.42039879e-01, -4.45672462e+00],
    [-1.06990503e-01, -1.38909109e-02,  1.49286383e-01,  7.86795690e+00],
    [ 1.93225466e-01,  1.91889701e-02,  3.06651133e-02,  1.96997049e+01]
])
intercept = np.array([
    14.296373741931472, 12.969070021138565, 9.473623617909242,
    8.424423453591807, 8.311382991983647, 7.856482295692187,
    6.9190453801055565, 5.493387660323357, -6.902783464414643,
    1.7680533760553654, -3.3749032964865906, -13.15566426896998,
    -19.95395768694729, -32.12453382191268
])

# Input data (X)
X = np.array([6,0,12,1])

# Calculate log-odds for each category
log_odds = np.dot(coefficients, X) + intercept

# Apply the softmax function to calculate probabilities
exp_log_odds = np.exp(log_odds - np.max(log_odds)) 
# For numerical stability
probabilities = exp_log_odds / np.sum(exp_log_odds)

print("Log-Odds:", log_odds)
print("Probabilities:", probabilities)

StatementMeta(, d9a8980a-0aa4-46a8-b8ca-95f2d30d8081, 6, Finished, Available, Finished)

Log-Odds: [ 14.94718425  15.23847353  11.41292045  11.56867116  11.0257505
  12.34248465 -11.43382529 -17.81439331  13.21203797 -14.73095655
  -3.71442859 -20.21991679 -10.93650721 -10.89749477]
Probabilities: [3.74315603e-01 5.00891033e-01 1.09226160e-02 1.27634575e-02
 7.41619069e-03 2.76714085e-02 1.30650495e-12 2.21342891e-15
 6.60196890e-02 4.83264779e-14 2.94172755e-09 1.99691694e-16
 2.14829322e-12 2.23375967e-12]
